In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
from HelperFunctions import *
from PtOccupancyFunctions import *
import pprint

In [ ]:
path = "/home/helge/Applications/matsim/matsim_results_ssd/PeriodicBC/8DrtCap/OwnIntermodalAccEgr/SteffenDrt/11_03_2021/"

In [ ]:
def getDeltaMax(path):
    return path.split("/")[-1].split("_")[-1]

def getDirs(directory):
    result = []
    sub_dirs = []
    sdirs = [
        sdir.path
        for sdir in
            os.scandir(directory)
        if sdir.is_dir() and "DeltaMax" in sdir.name
    ]

    for sdir in sorted(sdirs, key=lambda x: float(getDeltaMax(x))):
        subresult = {}
        subresult["root"] = sdir
        for root, subdirs, files in os.walk(sdir):
            for file in files:
                if file=="0.trips.csv.gz":
                    subresult["trips"] = os.path.join(root, file)
                if file=="0.drt_occupancy_time_profiles_drt.txt" and "unimodal" not in root:
                    subresult["drt_occupancy"] = os.path.join(root, file)
                if file=="trip_success.csv.gz":
                    subresult["trip_success"] = os.path.join(root, file)
                if file=="0.drt_trips_drt.csv":
                    subresult["drt_trips"] = os.path.join(root, file)
        result.append(subresult)
    
    return result

dirs = getDirs(path)
# pprint.pprint(dirs)

In [ ]:
def getTrips(paths, columns):
    path = paths["trips"]
    df = pd.read_csv(path, sep=";").loc[:,columns]
    df.set_index("person", inplace=True)
    df["trav_time"] = df["trav_time"].apply(timestmp2sec)
    df["wait_time"] = df["wait_time"].apply(timestmp2sec)
    return df

def getDrtOccupandyAndStandingFrac(paths, exclude_empty_vehicles):
    path = paths["drt_occupancy"]
    df = pd.read_csv(path, sep="\t")
    df["time"] = df["time"].apply(timestmphm2sec)
    drt_occ, drt_deviation = getAverageOcc(
        df.drop(columns="time"), exclude_empty_vehicles=exclude_empty_vehicles
    )
    drt_standing_frac = getStandingFraction(df.drop(columns="time"))
    return drt_occ, drt_standing_frac

def getTripSuccess(paths):
    path = paths["trip_success"]
    df = pd.read_csv(path, sep=";")
    return df

def getDrtTrips(paths):
    path = paths["drt_trips"]
    df = pd.read_csv(path, sep=";").loc[:, ["travelDistance_m", "travelTime"]]
    return df

In [ ]:
delta_max_list = np.empty(15)
requested_dists = np.empty(15)
drt_occupancies_non_empty = np.empty(15)
drt_occupancies_non_standing = np.empty(15)
serviced_fracts = np.empty(15)
travel_times = np.empty(15)
wait_times = np.empty(15)
cummulative_drt_travel_times = np.empty(15)
cummulative_drt_distance = np.empty(15)

In [ ]:
columns=["person", "euclidean_distance", "trav_time", "wait_time"]

for i,dic in enumerate(dirs):
    delta_max_list[i] = getDeltaMax(dic["root"])
    print('Delta Max: ', delta_max_list[i])
    
    trips = getTrips(dic, columns)
    drt_trips = getDrtTrips(dic)
    
    drt_occupancies_non_standing[i], _ = getDrtOccupandyAndStandingFrac(dic, False)
    drt_occupancies_non_empty[i], _ = getDrtOccupandyAndStandingFrac(dic, True)
    requested_dists[i] = trips['euclidean_distance'].mean()
    trip_succ = getTripSuccess(dic)
    serviced_fracts[i] = len(trip_succ[trip_succ["tripSuccess"]]) / len(trip_succ)
    travel_times[i] = trips["trav_time"].mean()
    wait_times[i] = trips["wait_time"].mean()
    cummulative_drt_travel_times[i] = drt_trips["travelTime"].sum()
    cummulative_drt_distance[i] = drt_trips["travelDistance_m"].sum()
    

In [ ]:
data = np.stack((requested_dists, drt_occupancies_non_empty, drt_occupancies_non_standing, serviced_fracts, travel_times, wait_times, cummulative_drt_travel_times, cummulative_drt_distance))
columns=['MeanServicedEuclideanDistance', 'NonEmptyDrtOccupancy', 'NonStandingDrtOccupancy', 'ServicedFraction', "TravelTimes", "WaitTimes", "CummulativeDrtTravelTimes", "CummulativeDrtDistance"]
df = pd.DataFrame(data=data.T, index=delta_max_list, columns=columns)
df.index.rename('DeltaMax', inplace=True)

In [ ]:
print(df_smart["CummulativeDrtDistance"]/df["CummulativeDrtDistance"])
# df_smart = df

In [ ]:
display(df)
save_path = os.path.join(path, 'DataAnalysis.csv')
print("Saving to ", save_path)
# df.to_csv(save_path)